In [ ]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import warnings
pd.set_option("display.max_colwidth", 250)
warnings.filterwarnings("ignore"

In [ ]:
table = str.maketrans("", "", punctuation)


#### Preprocessing the data
- Remove Punctuation
- Lower All the sentences
- Remove Punctuations


In [ ]:
data  = pd.read_csv("./summary.csv")

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [ ]:
def preprocess(input_list, table=table) :
    output_list = []
    for row in input_list:
        row_list = ""
        for contract_k, contract_v in contraction_mapping.items():
            row.replace(contract_k, contract_v)
        for word in row.split():
            # if word not in stop_words:
            word = word.translate(table)
            row_list += word.lower() + " "
        output_list.append(row_list.strip())
        
    return np.array(output_list)

In [ ]:
headlines = preprocess(data.headlines.to_list())
text = preprocess(data.text.to_list())

data.headlines = headlines
data.text = text
data.headlines = data.headlines.apply(lambda x: "_START " + x + " _END")

In [ ]:
for i in range(5):
    print("Review:",data['text'][i])
    print("Summary:",data['headlines'][i])
    print("\n")

In [ ]:
del headlines
del text

In [ ]:
text_word_count = []
summary_word_count = []

# populate the lists with sentence lengths
for i in data['text']:
      text_word_count.append(len(i.split()))

for i in data['headlines']:
      summary_word_count.append(len(i.split()))

length_df = pd.DataFrame({'text':text_word_count, 'summary':summary_word_count})
length_df.hist(bins = 30)
plt.show()

In [ ]:
train, test = train_test_split(data, test_size=0.3, random_state=42)

In [ ]:
def tokenizer(tok_object, x):
    all_ = x
    tk = tok_object
    tk.fit_on_texts(all_)
    return tk

x_tok = tokenizer(Tokenizer(), train.text.to_list())
y_tok = tokenizer(Tokenizer(), train.headlines.to_list())

In [ ]:
# TOkenize the train data
def tokenize(input_, pad_x_size:int = 70, pad_y_size:int = 20, x_tok=x_tok, y_tok=y_tok):
    
    y = input_.headlines.to_numpy()
    x = input_.text.to_numpy()
    
    headlines = y_tok.texts_to_sequences(np.array(y))
    text = x_tok.texts_to_sequences(np.array(x))
    
    headlines = pad_sequences(headlines, padding="post", maxlen=pad_y_size)
    text = pad_sequences(text, padding="post", maxlen=pad_x_size)
    
    return headlines, text

In [ ]:
headlines_train, text_train = tokenize(train)
headlines_test, text_test = tokenize(test)

In [ ]:
assert len(headlines_test) == len(text_test)
assert len(headlines_train) == len(text_train)

#### Build The model and train the dataset

In [ ]:
y_voc_size  =   len(y_tok.word_index) +1
x_voc_size  =   len(x_tok.word_index) +1

### Build the model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ( Embedding, LSTM, RepeatVector, TimeDistributed, 
                                    Dense, Input, Bidirectional, Concatenate)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from attention import AttentionLayer
from tensorflow.keras.utils import plot_model

In [ ]:
dim = 256

## Encoder With Lstm
inputs = Input(shape=(70,))
embedding = Embedding(x_voc_size, dim, trainable=True)(inputs)
#LSTM1
lstm1 = LSTM(dim, return_sequences=True, return_states=True)
encoder_output1, state_h1, state_c1 = lstm1(embedding)
# LSTM2
lstm1 = LSTM(dim, return_sequences=True, return_states=True)
encoder_output2, state_h2, state_c2 = lstm1(encoder_output1)
# LSTM3
lstm1 = LSTM(dim, return_sequences=True, return_states=True)
encoder_outputs, state_h, state_c = lstm1(encoder_output2)

# Decoder Inputs
inputs_decoder = Input( shape=(None,))
decoder_embedding_layer = Embedding(x_voc_size, dim, trainable=True)
decoder_outputs_embedding = decoder_embedding_layer(inputs_decoder)
# LSTM Decoder
decoder_lstm = LSTM( dim, return_sequences=True, return_states=True)
decoder_outputs, decoder_fwd_state, decoder_bwd_state = decoder_lstm(
    decoder_outputs_embedding, initial_state=[state_h, state_c])
#Attention, layer, 
attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs]) 
# Concatenate the layers
decoder_concat_input = Concatenate(axis=-1, name="concat_layer")[decoder_outputs, attn_out]
# Dense Output layer
dense_decoder = TimeDistributed(Dense(y_voc_size, activation="softmax"))
decoder_outputs = dense_decoder(decoder_concat_input)

model = Model([inputs, inputs_decoder], decoder_outputs)
model.summary()


plot_model(model)

In [ ]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer ="rmsprop")
es = EarlyStopping(monitor="val_loss", mode="min", verbose=1)
checkpoint = ModelCheckpoint("./temp/model_best.h5", save_best_only= True)

In [ ]:
model.fit(np.array(text_train), np.array(headlines_train),validation_split=.25 ,epochs=128)